In [1]:
from treelib import Tree, Node
from treelib.plugins import export_to_dot
from polars import DataFrame
from typing import (
    Optional,
    Iterable,
    Union,
    Set,
    Literal,
    Callable,
    Generator,
    Sequence,
)
from dataclasses import dataclass
from operator import itemgetter, attrgetter, add, ne, contains, or_
from itertools import (
    repeat,
    accumulate,
    dropwhile,
    chain,
    groupby,
    starmap,
    filterfalse,
    dropwhile,
)
from functools import reduce, partial, total_ordering
from random import random, randint, choice, choices, sample
from collections import Counter, defaultdict, OrderedDict
from math import ceil
from enum import Enum
from copy import deepcopy

In [2]:
class VarEnum(Enum):
    FORBIDDEN = "FORBIDDEN"
    FREE = "FREE"


InputVarValue = Literal[VarEnum.FREE] | int
OutputVarValue = Literal[VarEnum.FORBIDDEN, VarEnum.FREE] | int


@dataclass(frozen=True, repr=False)
class Cube:
    input_vars: tuple[InputVarValue]
    output_vars: tuple[OutputVarValue]
    source: str = "unknown"

    def __repr__(self):
        return f"{format_input_vars(self.input_vars)} | {format_output_vars(self.output_vars)} | {self.source}"

    def __str__(self):
        return self.__repr__()

    def to_global(self, output_var_index, source="unknown"):
        return Cube(
            input_vars=local_to_global_input_var_list(
                output_var_index, self.input_vars
            ),
            output_vars=self.output_vars,
            source=source,
        )

    def with_replacements(
        self,
        input_vars: dict[int, InputVarValue] = dict(),
        output_vars: dict[int, OutputVarValue] = dict(),
        source: str = "unknown",
    ):
        return Cube(
            input_vars=tuple(
                starmap(
                    lambda i, x: x if i not in input_vars else input_vars[i],
                    enumerate(self.input_vars),
                )
            ),
            output_vars=tuple(
                starmap(
                    lambda i, x: x if i not in output_vars else output_vars[i],
                    enumerate(self.output_vars),
                )
            ),
            source=source,
        )


@dataclass(frozen=True)
class AddItems:
    iterable: Iterable[str]


@dataclass(frozen=True)
class PopItems:
    set: Set[str]

In [3]:
def format_input_vars(input_vars: tuple[InputVarValue]):
    return " ".join(
        list(map(lambda x: "x" if x == VarEnum.FREE else str(x), input_vars))
    )


def format_output_vars(output_vars: tuple[OutputVarValue]):
    return " ".join(
        list(
            map(
                lambda y: "\u0305y"
                if y == VarEnum.FORBIDDEN
                else "y"
                if y == VarEnum.FREE
                else str(y),
                output_vars,
            )
        )
    )

def format_input_output_vars(input_vars, output_vars):
    return f"{format_input_vars(input_vars)} | {format_output_vars(output_vars)}"

In [4]:
augmentable_list: list[str] = []


def augmentable_generator() -> (
    Generator[Optional[str], Optional[AddItems | PopItems], None]
):
    global augmentable_list
    while augmentable_list:
        command: Optional[AddItems | PopItems] = yield augmentable_list.pop(0)
        if command is not None:
            yield None
            if isinstance(command, AddItems):
                for new_item in command.iterable:
                    augmentable_list.insert(randint(0, len(augmentable_list)), new_item)
            elif isinstance(command, PopItems):
                len_before = len(augmentable_list)
                in_set: Callable[[str], bool] = partial(contains, command.set)
                augmentable_list = list(filterfalse(in_set, augmentable_list))
                len_after = len(augmentable_list)


def random_unbound_index(input_vars: list[InputVarValue]) -> int | None:
    """Псевдослучайный выбор индекса входной несвязанной переменной"""
    matching_indices = list(
        map(
            itemgetter(0), filter(lambda x: x[1] == VarEnum.FREE, enumerate(input_vars))
        )
    )
    if matching_indices:
        return choice(matching_indices)
    return None


def numerate_input_vars(
    input_vars: tuple[InputVarValue], output_var_index: int
) -> Iterable[tuple[int, InputVarValue]]:
    """Нумерация индексов входных переменных атрибута"""
    return zip(cs[output_var_index], input_vars)


def extend_list(l, target_size, filler):
    return l + [filler] * (target_size - len(l))


def local_to_global_input_var_index(output_var_index: int, local_index: int) -> int:
    return cs[output_var_index][local_index]


def has_last_input_var(output_var_index):
    return (l - 1) in cs[output_var_index]


def local_to_global_input_var_list(
    output_var_index: int, input_vars: tuple[InputVarValue]
) -> tuple[InputVarValue]:
    filler_type = tuple[tuple[InputVarValue], int]
    free_filler_value: tuple[InputVarValue] = (VarEnum.FREE,)

    def space_filler(acc: filler_type, x: tuple[int, InputVarValue]) -> filler_type:
        return (
            (*acc[0], *(free_filler_value * (x[0] - acc[-1] - 1)), x[1]),
            x[0],
        )

    numerated_vars: Iterable[tuple[int, InputVarValue]] = chain(
        numerate_input_vars(input_vars, output_var_index),
        [
            (
                l - 1,
                VarEnum.FREE,
            )
        ]
        if not has_last_input_var(output_var_index)
        else [],
    )
    reduce_init: tuple[tuple[InputVarValue], int] = (
        tuple(),
        -1,
    )
    # type: ignore
    result: tuple[InputVarValue] = list(
        reduce(space_filler, numerated_vars, reduce_init)
    )[
        0
    ]  # type: ignore
    assert -1 not in result
    assert type(result) is tuple
    if len(result) != l:
        print(result)
        print(
            f"output_var_index: {output_var_index}\tinput_vars: {input_vars}\tlen(result): {len(result)} has_last_input_var: {has_last_input_var(output_var_index)}"
        )
        raise Exception()
    return result


def needed_j(connected_probabilities):
    """
    Псевдослучайный датчик ИС, на вход принимает
    таблицу вероятностей p для каждого j, возвращает
    j - число требуемых связанных переменных
    """
    return choices(range(l + 1), connected_probabilities)[0]


def add_child(parent: Node, tree: Tree, child_input_vars: tuple[InputVarValue]) -> Node:
    return tree.create_node(
        tag=format_input_output_vars(child_input_vars, parent.data.output_vars),
        data=Cube(
            input_vars=child_input_vars,
            output_vars=parent.data.output_vars,
        ),
        parent=parent.identifier,
    ).identifier


def create_initial_cube(output_var_index, l, tree):
    input_vars = tuple([VarEnum.FREE] * len(cs[output_var_index]))
    output_vars=(
        *repeat(VarEnum.FORBIDDEN, output_var_index),
        VarEnum.FREE,
        *repeat(VarEnum.FORBIDDEN, l - output_var_index - 1),
    )
    return tree.create_node(
        tag=format_input_output_vars(input_vars, output_vars),
        data=Cube(
            input_vars=input_vars,
            output_vars=output_vars,
        ),
    ).identifier


def random_output_value_by_index(output_var_index):
    return choice(range(output_var_lens[output_var_index]))


def fill_output_vars(cube: Cube):
    return Cube(
        input_vars=cube.input_vars,
        output_vars=tuple(
            starmap(
                lambda i, x: random_output_value_by_index(i)
                if x == VarEnum.FREE
                else x,
                enumerate(cube.output_vars),
            )
        ),
        source=cube.source,
    )


def gamma_transformation(output_var_index, cube_input_vars, var_lens):
    var_index = random_unbound_index(cube_input_vars)
    if var_index is not None:
        var_len = var_lens[local_to_global_input_var_index(output_var_index, var_index)]
        (vars_before, vars_after) = itemgetter(
            slice(0, var_index), slice(var_index + 1, m)
        )(cube_input_vars)
        return map(
            lambda var: (
                *vars_before,
                var,
                *vars_after,
            ),
            range(var_len),
        )
    return []


def input_vars_count(output_var_index):
    return len(cs[output_var_index])


def num_connected_vars(input_vars: tuple[InputVarValue]):
    return len(list(filter(lambda x: x != VarEnum.FREE, input_vars)))


def max_input_vars_reached(output_var_index: int, input_vars: tuple[InputVarValue]):
    return num_connected_vars(input_vars) >= max_connected_probability_index()


def max_connected_probability_index():
    return list(filter(lambda pair: pair[1] > 0, enumerate(connected_probabilities)))[
        -1
    ][0]


def cube_connected_vars(cube: Cube):
    return num_connected_vars(cube.input_vars)


def count_ross(reduced_set):
    N = len(reduced_set)
    # μ
    mus = defaultdict(
        lambda: 0, Counter(map(lambda x: num_connected_vars(x.input_vars), reduced_set))
    )
    # μ'
    muss = list(
        map(lambda n: max(0, (N + 1) * connected_probabilities[n] - mus[n]), range(l))
    )
    # сумма μ'
    mussum = sum(muss)
    # ρ'
    ross = list(map(lambda n: muss[n] / mussum, range(l)))
    return ross


def take_random_members_from_set(st, probability):
    if probability:
        return sample(st, ceil(len(st) * probability))
    return []


def reduce_set_to_probability(reduced_set):
    ross = count_ross(reduced_set)
    groups = defaultdict(
        lambda: [],
        starmap(
            lambda i, x: (i, list(map(itemgetter(1), x))),
            groupby(
                sorted(
                    map(lambda x: (cube_connected_vars(x), x), reduced_set),
                    key=itemgetter(0),
                ),
                key=itemgetter(0),
            ),
        ),
    )
    res_groups = dict(
        map(
            lambda i: (i, take_random_members_from_set(groups[i], ross[i])),
            range(len(ross)),
        )
    )
    # print('res_groups', res_groups)
    max_index, max_group = max(res_groups.items(), key=lambda x: len(x[1]))
    max_count = len(max_group)
    max_probability = connected_probabilities[max_index]
    relative_probabilities = list(
        map(lambda x: x / max_probability, connected_probabilities)
    )
    fill_indices = list(
        filter(
            lambda pair: pair[1] > 0 and len(res_groups[pair[0]]) == 0,
            enumerate(relative_probabilities),
        )
    )
    fill_groups = dict(
        starmap(
            lambda i, prob: (i, sample(groups[i], ceil(prob * max_count))), fill_indices
        )
    )
    res_groups = res_groups | fill_groups
    return tuple(sum(map(itemgetter(1), res_groups.items()), []))

## Задание входных параметров программы

In [5]:
l = 10  # число атрибутов решения
m = 10  # число атрибутов условия
i_max = 1000  # число строк в результирующей ТР
x_review_period = 100  # период пересмотра значения x
x = 0.45  # вероятность того, что входная координата является связной

## Задание параметров ИС

In [6]:
var_lens: list[int] = list(repeat(6, l))  # Количество значений каждой переменной
output_var_lens: list[int] = list(repeat(5, m))
connected_probabilities: list[float] = [
    0,
    0,
    0.32,
    0.38,
    0.3,
    *repeat(0, 6),
]  # Распределение вероятностей количества связанных переменных
# Множество входных переменных, от которых зависит хотя бы одна функция
cs: list[list[int]] = [
    [*range(5), *range(6, 10)],
    [*range(4), *range(5, 7)],
    [*range(2, 10)],
    [*range(1, 8), 9],
    [*range(1, 3), *range(4, 10)],
    [*range(2), *range(4, 9)],
    [2, *range(5, 8)],
    [0, 4, 7, 8],
    [*range(3), 4],
    [*range(4), *range(5, 7), 8],
]

## Этап 1: Формирование входной части многовыходных кубов

In [7]:
target_set = []
popped_items = set()
for output_var_index in range(l):
    global augmentable_list
    tree = Tree()
    augmentable_list = [create_initial_cube(output_var_index, l, tree)]
    bound_target_set = []
    gen = augmentable_generator()
    for node_id in gen:
        node = tree.get_node(node_id)
        j = needed_j(connected_probabilities)
        connected_vars = num_connected_vars(node.data.input_vars)
        if connected_vars == j:
            bound_target_set.append(
                node.data.to_global(output_var_index, source="matched")
            )
        else:
            mivr = max_input_vars_reached(output_var_index, node.data.input_vars)
            if connected_vars < j and not mivr:  # not mivr
                gen.send(
                    AddItems(
                        iterable=map(
                            partial(add_child, node, tree),
                            gamma_transformation(
                                output_var_index, node.data.input_vars, var_lens
                            ),
                        )
                    )
                )
            elif connected_vars > j:
                parent_node = tree.parent(node_id)
                if parent_node:
                    new_popped_items = set(
                        map(
                            attrgetter("identifier"),
                            tree.subtree(parent_node.identifier).all_nodes(),
                        )
                    )
                    # isect = (popped_items & new_popped_items)
                    # if (len(isect) > 0):
                    #     print('intersection' , isect)
                    #     raise Exception()
                    popped_items |= new_popped_items
                    gen.send(PopItems(set=set(new_popped_items)))
                bound_target_set.append(
                    parent_node.data.to_global(output_var_index, source="parent")
                )
            # elif (num_connected_vars(node.data.input_vars) == j):
            #     target_set.append(node.data.to_global(output_var_index, source="max input vars"))
    # export_to_dot(tree, 'tree.dot')
    # tree.show()
    # Пересчёт целевого множества для одной выходной координаты
    bound_target_set = reduce_set_to_probability(bound_target_set)
    target_set.extend(bound_target_set)

In [8]:
# Печать дерева
st = tree.subtree(tree.root)
for node in st.children(st.root)[1:]:
    st.remove_subtree(node.identifier)
for i, node in enumerate(st.children(st.children(st.root)[0].identifier)):
    if i != 4:
        st.remove_subtree(node.identifier)
st.to_graphviz('tree.dot', shape='rect')

In [9]:
# Результирующее множество кубов, распределённое по количеству связанных переменных
print(Counter(map(lambda x: num_connected_vars(x.input_vars), target_set)))

Counter({3: 88, 2: 75, 4: 65})


## Этап 2

In [10]:
# Пересчёт целевого множества для всех выходных координат
target_set = reduce_set_to_probability(target_set)

## Заполнение выходных координат

In [11]:
target_set = list(map(fill_output_vars, target_set))

In [12]:
# Количество кубов по источникам происхождения - parent - обратная λ-трансформация, matched - прямая λ-трансформация
print(Counter(map(lambda x: (x.source, num_connected_vars(x.input_vars)), target_set)))

Counter({('matched', 4): 65, ('matched', 3): 48, ('parent', 2): 46, ('parent', 3): 35, ('matched', 2): 24})


In [13]:
# Целевое распределение кубов
needed_distribution = tuple(zip(range(l + 1), connected_probabilities))

In [14]:
# Результирующее распределение кубов по количеству связанных переменных
result_distribution = tuple(
    starmap(
        lambda i, x: (i, x / len(target_set)),
        Counter(map(lambda c: num_connected_vars(c.input_vars), target_set)).items(),
    )
)

In [15]:
# Вывод всего целевого множества
print("\n".join(map(str, target_set)))

3 x x 2 x x x x x x | 0 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | matched
5 x x x x 0 x x x x | ̅y ̅y ̅y ̅y ̅y 2 ̅y ̅y ̅y ̅y | parent
x x x 1 x x x x 2 x | 0 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
0 x x x x x x 1 x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y 0 ̅y ̅y | parent
x x x x x 3 2 x x x | ̅y ̅y ̅y ̅y ̅y ̅y 3 ̅y ̅y ̅y | matched
5 x x x 3 x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 3 ̅y | parent
x 3 x x x 4 x x x x | ̅y 4 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
4 x x x x x x x 1 x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y ̅y | matched
x x x x x x 1 x x 2 | ̅y ̅y ̅y 4 ̅y ̅y ̅y ̅y ̅y ̅y | matched
x x x 1 x x x x 1 x | 4 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
x 0 x x x 1 x x x x | ̅y 3 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
5 x x x 2 x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y | matched
2 0 x x x x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 4 ̅y | matched
x x x x x 2 0 x x x | ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y ̅y ̅y | matched
x 2 x x x 3 x x x x | ̅y 1 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | matched
4 2 x x x x x x x x | ̅y ̅y ̅y ̅y ̅y 0 ̅y ̅y ̅y ̅y | parent
2 x x 0 x x x x x x | ̅y ̅y ̅y ̅

In [16]:
mixed_cubes = target_set

In [17]:
def cubes_with_no_output_var_conflicts(cube):
    return lambda c: map(
        lambda pair: len(pair) == 1
        or VarEnum.FORBIDDEN in pair
        or VarEnum.FREE in pair,
        map(set, zip(cube.output_vars, c.output_vars)),
    )


def count_var_values(all_cubes, var_lens, output_var_lens):
    initial_counters = tuple(
        map(
            lambda x: tuple(
                map(
                    lambda var_len: dict(map(lambda i: (i, 0), range(var_len))),
                    x
                )
            ),
            (var_lens, output_var_lens),
        )
    )
    return tuple(
        map(
            lambda grp: tuple(map(Counter, starmap(or_, zip(*grp)))),
        zip(
        initial_counters,
        map(
            lambda grp: list(map(lambda values: dict(Counter(values)), grp)),
            reduce(
                lambda acc, x: list(
                    starmap(
                        lambda accvars, xvars: list(starmap(add, zip(accvars, xvars))),
                        zip(acc, x),
                    )
                ),
                map(
                    lambda cube: (
                        list(map(lambda x: [x], cube.input_vars)),
                        list(map(lambda x: [x], cube.output_vars)),
                    ),
                    all_cubes,
                ),
            ),
        )
        )
        )
    )


a = count_var_values(
    [
        Cube(input_vars=[1, 2, 3], output_vars=[4, 5, 6]),
        Cube(input_vars=[7, 8, 9], output_vars=[10, 11, 12]),
    ],
    (3, 3, 3),
    (4, 4, 4),
)
print(a)

((Counter({1: 1, 7: 1, 0: 0, 2: 0}), Counter({2: 1, 8: 1, 0: 0, 1: 0}), Counter({3: 1, 9: 1, 0: 0, 1: 0, 2: 0})), (Counter({4: 1, 10: 1, 0: 0, 1: 0, 2: 0, 3: 0}), Counter({5: 1, 11: 1, 0: 0, 1: 0, 2: 0, 3: 0}), Counter({6: 1, 12: 1, 0: 0, 1: 0, 2: 0, 3: 0})))


In [18]:
assert len(target_set) == len(mixed_cubes)
# Вывод промежуточного целевого множества
print("\n".join(map(str, mixed_cubes)))

3 x x 2 x x x x x x | 0 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | matched
5 x x x x 0 x x x x | ̅y ̅y ̅y ̅y ̅y 2 ̅y ̅y ̅y ̅y | parent
x x x 1 x x x x 2 x | 0 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
0 x x x x x x 1 x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y 0 ̅y ̅y | parent
x x x x x 3 2 x x x | ̅y ̅y ̅y ̅y ̅y ̅y 3 ̅y ̅y ̅y | matched
5 x x x 3 x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 3 ̅y | parent
x 3 x x x 4 x x x x | ̅y 4 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
4 x x x x x x x 1 x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y ̅y | matched
x x x x x x 1 x x 2 | ̅y ̅y ̅y 4 ̅y ̅y ̅y ̅y ̅y ̅y | matched
x x x 1 x x x x 1 x | 4 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
x 0 x x x 1 x x x x | ̅y 3 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | parent
5 x x x 2 x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y | matched
2 0 x x x x x x x x | ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y 4 ̅y | matched
x x x x x 2 0 x x x | ̅y ̅y ̅y ̅y ̅y ̅y 1 ̅y ̅y ̅y | matched
x 2 x x x 3 x x x x | ̅y 1 ̅y ̅y ̅y ̅y ̅y ̅y ̅y ̅y | matched
4 2 x x x x x x x x | ̅y ̅y ̅y ̅y ̅y 0 ̅y ̅y ̅y ̅y | parent
2 x x 0 x x x x x x | ̅y ̅y ̅y ̅

In [19]:
def fill_cube_with_input_vars(cube, all_cubes):
    for i, var in filter(lambda x: type(x[1]) is VarEnum, enumerate(cube.input_vars)):
        output_var_conflicting_cubes = list(
            filterfalse(cubes_with_no_output_var_conflicts(cube), all_cubes)
        )
        if len(output_var_conflicting_cubes) == 0:
            output_var_conflicting_cubes = all_cubes
        input_vars, _ = count_var_values(output_var_conflicting_cubes, var_lens, output_var_lens)
        least_common_input_var = next(
            filter(
                lambda x: type(x) is not VarEnum,
                map(itemgetter(0), reversed(input_vars[i].most_common())),
            )
        )
        cube = cube.with_replacements(
            input_vars={i: least_common_input_var}, source="fill"
        )
    return cube


def fill_cube_with_output_vars(all_cubes):
    def reducer(acc, cube):
        for i, var in filter(
            lambda x: type(x[1]) is VarEnum, enumerate(cube.output_vars)
        ):
            output_var_conflicting_cubes = list(
                filterfalse(cubes_with_no_output_var_conflicts(cube), all_cubes)
            )
            if len(output_var_conflicting_cubes) == 0:
                output_var_conflicting_cubes = all_cubes
            _, output_vars = count_var_values(output_var_conflicting_cubes, var_lens, output_var_lens)
            new_cube = next(
                filter(
                    lambda c: c not in acc,
                    map(
                        lambda var: cube.with_replacements(
                            output_vars={i: var}, source="fill_output"
                        ),
                        filter(
                            lambda x: type(x) is not VarEnum,
                            map(itemgetter(0), reversed(output_vars[i].most_common())),
                        ),
                    ),
                ),
                None,
            )
            if new_cube is not None:
                cube = new_cube
            else:
                cube = None
                break
        if cube:
            return [cube, *acc]
        else:
            return acc

    return reducer


# fill_cube_with_output_vars(mixed_cubes)
filled_cubes = list(
    reduce(
        fill_cube_with_output_vars(mixed_cubes),
        map(lambda cube: fill_cube_with_input_vars(cube, mixed_cubes), mixed_cubes),
        [],
    )
)
print("Результирующая таблица решений:\n", "\n".join(map(str, filled_cubes)))

Результирующая таблица решений:
 5 5 0 2 3 5 5 0 1 5 | 4 0 4 1 0 4 2 2 2 4 | fill_output
5 1 0 2 3 4 3 1 1 5 | 4 1 4 1 0 4 3 2 2 4 | fill_output
5 1 5 2 3 4 3 3 1 5 | 4 1 3 1 0 4 2 2 2 4 | fill_output
5 1 1 1 3 5 0 3 4 5 | 2 1 4 1 0 4 2 2 2 4 | fill_output
5 1 2 3 4 5 0 0 1 0 | 0 1 4 1 0 4 2 2 2 4 | fill_output
4 2 5 2 2 5 0 2 1 5 | 4 1 4 1 0 4 2 2 2 4 | fill_output
5 4 5 3 3 4 0 0 1 5 | 4 4 4 1 0 4 2 2 2 4 | fill_output
5 1 5 3 4 5 0 0 1 3 | 3 1 4 1 0 4 2 2 2 4 | fill_output
2 0 2 4 3 5 0 0 1 5 | 4 1 4 1 0 4 2 2 2 3 | fill_output
5 1 5 2 3 3 4 4 1 5 | 4 1 4 1 0 4 1 2 2 4 | fill_output
5 1 0 2 3 3 4 4 1 5 | 4 1 4 1 0 4 4 2 2 4 | fill_output
1 1 5 2 4 4 0 5 1 5 | 4 1 4 1 0 3 2 2 2 4 | fill_output
2 4 5 5 3 4 0 0 1 5 | 4 4 4 1 0 4 2 2 2 4 | fill_output
0 5 5 2 3 0 3 0 1 5 | 4 1 4 1 0 4 2 2 2 4 | fill_output
0 3 5 2 3 5 0 1 3 5 | 4 1 4 1 0 2 2 2 2 4 | fill_output
3 2 5 2 3 2 5 0 1 5 | 4 2 4 1 0 4 2 2 2 4 | fill_output
5 1 4 2 3 3 4 1 1 5 | 4 1 4 1 0 4 0 2 2 4 | fill_output
5 1 1 2 3 5 5 3

In [20]:
df = DataFrame(
    reduce(
        lambda acc, x: dict(map(lambda key: (key, [*acc[key], x[key]]), x.keys())),
        starmap(
            lambda i, cube: (
                {"No": i}
                | dict(starmap(lambda i, v: (f"i{i}", v), enumerate(cube.input_vars)))
                | dict(starmap(lambda i, v: (f"o{i}", v), enumerate(cube.output_vars)))
            ),
            enumerate(filled_cubes),
        ),
        {"No": []}
        | dict(
            chain(
                map(lambda i: (f"i{i}", []), range(m)),
                map(lambda i: (f"o{i}", []), range(l)),
            )
        ),
    )
)
df.write_csv("solution_table.csv")

In [21]:
print(len(set(filled_cubes)))
print("Целевое распределение:", needed_distribution)
print("Реальное распределене:", result_distribution)

218
Целевое распределение: ((0, 0), (1, 0), (2, 0.32), (3, 0.38), (4, 0.3), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0))
Реальное распределене: ((2, 0.3211009174311927), (3, 0.38073394495412843), (4, 0.2981651376146789))
